In [3]:
import pandas as pd
import pickle 


filename = 'vacancies.pickle'
with open("../data/better_preprocessed_TRAIN_lda_data", 'rb') as f:
  train = pickle.load(f)
# train  = pd.read_csv("../data/better_preprocessed_TRAIN_lda_data.csv")
# test  = pd.read_csv("/data/test_processed.csv")

In [4]:
with open("../data/better_preprocessed_TEST_lda_data", 'rb') as f:
  test = pickle.load(f)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=20000) # не забываем про n-грамы и фиксацию размера словаря

train_vectorized = vectorizer.fit_transform(train["text"].values)
test_vectorized = vectorizer.transform(test["text"].values)



#LDA

Latent Dirichlet Allocation - Скрытое распределение Дирихле

In [6]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model_sklearn = LatentDirichletAllocation(n_components=15, random_state=42)
lda_model_sklearn.fit(train_vectorized)

lda_topics_test_sklearn = lda_model_sklearn.transform(test_vectorized)

In [7]:
print(lda_topics_test_sklearn)

[[1.23916066e-04 1.23915953e-04 6.27722841e-02 ... 1.23916073e-04
  1.23915996e-04 1.52317183e-01]
 [1.74065014e-04 2.47414673e-02 7.03254549e-01 ... 1.74064798e-04
  1.74064802e-04 1.75465117e-01]
 [2.22965744e-04 2.22965718e-04 2.22966081e-04 ... 2.22965885e-04
  2.22965815e-04 2.22965853e-04]
 ...
 [1.30719325e-04 1.02397323e-01 2.88630983e-01 ... 1.30719151e-04
  1.30719349e-04 1.30719283e-04]
 [4.79972889e-02 1.02419748e-01 1.15740993e-04 ... 1.15740973e-04
  1.15740993e-04 1.15741102e-04]
 [5.42006671e-04 7.40151716e-02 5.42007197e-04 ... 5.42006644e-04
  5.42006472e-04 5.59199847e-01]]


In [8]:
vocab = vectorizer.get_feature_names_out()
print(vocab, len(vocab))

['абакумов' 'аббас' 'аббревиатура' ... 'ёлочный' 'ёмкий' 'ёмкость'] 20000


In [9]:
# .components_ - количество раз, когда слово было назначено теме

for idx, topic in enumerate(lda_model_sklearn.components_):
    # .argsort() индексы, которые отсортируют массив topic по возрастанию
    # [:-top_words_count - 1:-1] - необходимо взять последние индексы из отсортированного массива
    top_words = [vocab[i] for i in topic.argsort()[: -11:-1]]
    print(f"Тема {idx+1}: {top_words}")

Тема 1: ['свой', 'весь', 'год', 'песня', 'новый', 'очень', 'концерт', 'стать', 'время', 'ещё']
Тема 2: ['год', 'весь', 'свой', 'человек', 'музей', 'церковь', 'памятник', 'время', 'храм', 'наш']
Тема 3: ['россия', 'военный', 'российский', 'год', 'украина', 'страна', 'сша', 'мочь', 'свой', 'весь']
Тема 4: ['суд', 'дело', 'свой', 'закон', 'мочь', 'год', 'выбор', 'весь', 'депутат', 'партия']
Тема 5: ['самолёт', 'мочь', 'год', 'весь', 'наш', 'ещё', 'продукт', 'время', 'полёт', 'человек']
Тема 6: ['вода', 'человек', 'весь', 'год', 'животное', 'день', 'собака', 'мочь', 'ещё', 'время']
Тема 7: ['человек', 'фильм', 'очень', 'весь', 'свой', 'театр', 'год', 'мочь', 'жизнь', 'говорить']
Тема 8: ['весь', 'свой', 'год', 'ребёнок', 'человек', 'мочь', 'говорить', 'знать', 'ещё', 'сказать']
Тема 9: ['человек', 'врач', 'ребёнок', 'год', 'мочь', 'весь', 'медицинский', 'пациент', 'помощь', 'лечение']
Тема 10: ['год', 'москва', 'дом', 'весь', 'город', 'новый', 'рубль', 'строительство', 'квартира', 'район']

In [10]:
doc_topic = lda_model_sklearn.transform(test_vectorized)

for n in range(10):
    topic_doc = doc_topic[n].argmax()
    print(f"Документ {n+1} - Тема:{topic_doc}")

Документ 1 - Тема:4
Документ 2 - Тема:2
Документ 3 - Тема:8
Документ 4 - Тема:0
Документ 5 - Тема:3
Документ 6 - Тема:7
Документ 7 - Тема:11
Документ 8 - Тема:3
Документ 9 - Тема:12
Документ 10 - Тема:4


--------------------------------------------------

In [11]:
from gensim.corpora.dictionary import Dictionary
from gensim.matutils import Sparse2Corpus

#  documents_columns=False - строки  представляют собой документы, а столбцы — это уникальные слова
gensim_corpus_train = Sparse2Corpus(train_vectorized, documents_columns=False) # id слова и частота
# Выводим содержимое корпуса
# for doc in gensim_corpus_train:
#     print(doc)
print(gensim_corpus_train)
id2word={i: word for word, i in dict}

gensim_dictionary = Dictionary.from_corpus(gensim_corpus_train, id2word=id2word)
print(gensim_dictionary)


TypeError: 'type' object is not iterable

In [ ]:
from gensim.models.ldamodel import LdaModel

In [ ]:
lda_model_gensim = LdaModel(corpus=gensim_corpus_train, id2word=gensim_dictionary, num_topics=15)

In [ ]:
import re

lda_topics_gensim = lda_model_gensim.print_topics(num_topics=15, num_words=10)
for idx, topic in lda_topics_gensim:
    # Используем регулярные выражения для извлечения слов
    words = re.findall(r'"(.*?)"', topic)
    print(f"Тема {idx}: {words}")

Тема 0: ['это', 'который', 'год', 'всё', 'мочь', 'свой', 'человек', 'весь', 'ещё', 'первый']
Тема 1: ['это', 'год', 'который', 'мочь', 'свой', 'всё', 'весь', 'время', 'человек', 'говорить']
Тема 2: ['это', 'который', 'год', 'всё', 'человек', 'свой', 'весь', 'мочь', 'время', 'ещё']
Тема 3: ['это', 'всё', 'человек', 'свой', 'который', 'год', 'весь', 'мочь', 'время', 'ребёнок']
Тема 4: ['год', 'это', 'всё', 'который', 'свой', 'человек', 'мочь', 'весь', 'россия', 'наш']
Тема 5: ['это', 'год', 'который', 'человек', 'всё', 'мочь', 'весь', 'свой', 'время', 'говорить']
Тема 6: ['это', 'который', 'год', 'всё', 'человек', 'весь', 'свой', 'россия', 'мочь', 'время']
Тема 7: ['год', 'это', 'который', 'всё', 'весь', 'человек', 'свой', 'мочь', 'россия', 'наш']
Тема 8: ['это', 'год', 'который', 'свой', 'всё', 'мочь', 'наш', 'время', 'россия', 'весь']
Тема 9: ['это', 'всё', 'который', 'свой', 'мочь', 'весь', 'человек', 'год', 'россия', 'очень']
Тема 10: ['это', 'который', 'год', 'свой', 'всё', 'человек

# NMF

Non-Negative Matrix Factorization (NMF) - Неотрицательная матричная факторизация

In [ ]:
from sklearn.decomposition import NMF

nmf_model_sklearn = NMF(n_components=15, random_state=42)
nmf_model_sklearn.fit(train_vectorized)

nmf_topics_test_sklearn = lda_model_sklearn.transform(test_vectorized)

In [ ]:
for idx, topic in enumerate(nmf_model_sklearn.components_):
    # .argsort() индексы, которые отсортируют массив topic по возрастанию
    # [:-top_words_count - 1:-1] - необходимо взять последние индексы из отсортированного массива
    top_words = [vocab[i] for i in topic.argsort()[: -11:-1]]
    print(f"Тема {idx+1}: {top_words}")

Тема 1: ['это', 'человек', 'всё', 'мочь', 'который', 'говорить', 'просто', 'сказать', 'очень', 'знать']
Тема 2: ['год', 'это', 'весь', 'который', 'первый', 'наш', 'новый', 'москва', 'тысяча', 'время']
Тема 3: ['ребёнок', 'родитель', 'школа', 'всё', 'семья', 'свой', 'это', 'мама', 'сын', 'жить']
Тема 4: ['свой', 'всё', 'год', 'жизнь', 'весь', 'стать', 'очень', 'ещё', 'время', 'первый']
Тема 5: ['лотерея', 'государственный', 'проведение', 'срок', 'всероссийский', 'рубль', 'билет', 'русский', 'лото', 'бестиражный']
Тема 6: ['который', 'это', 'страна', 'россия', 'свой', 'власть', 'президент', 'российский', 'наш', 'сша']
Тема 7: ['который', 'это', 'памятник', 'здание', 'объект', 'всё', 'работа', 'исторический', 'наш', 'должный']
Тема 8: ['всё', 'очередь', 'весь', 'ещё', 'мочь', 'человек', 'это', 'центр', 'наш', 'документ']
Тема 9: ['спортсмен', 'сборная', 'это', 'слепой', 'который', 'инвалид', 'россия', 'игра', 'спорт', 'тренер']
Тема 10: ['квартира', 'город', 'жильё', 'цена', 'год', 'москв

In [ ]:
from gensim.models import Nmf

In [ ]:
Nmf_model_gensim = Nmf(corpus=gensim_corpus_train, id2word=gensim_dictionary, num_topics=15)

In [ ]:
import re

nmf_topics_gensim = Nmf_model_gensim.print_topics(num_topics=15, num_words=10)
for idx, topic in nmf_topics_gensim:
    # Используем регулярные выражения для извлечения слов
    words = re.findall(r'"(.*?)"', topic)
    print(f"Тема {idx}: {words}")

Тема 0: ['свой', 'год', 'это', 'песня', 'наш', 'время', 'стать', 'фильм', 'российский', 'случай']
Тема 1: ['который', 'мочь', 'свой', 'россия', 'российский', 'наш', 'никто', 'партия', 'театр', 'вопрос']
Тема 2: ['это', 'всё', 'ребёнок', 'человек', 'мочь', 'свой', 'очень', 'просто', 'школа', 'родитель']
Тема 3: ['свой', 'год', 'ребёнок', 'дело', 'всё', 'дом', 'страна', 'семья', 'сын', 'весь']
Тема 4: ['дело', 'говорить', 'день', 'ещё', 'сказать', 'время', 'мочь', 'первый', 'наталья', 'всё']
Тема 5: ['россия', 'который', 'москва', 'новый', 'мочь', 'год', 'власть', 'это', 'партия', 'место']
Тема 6: ['это', 'который', 'год', 'область', 'человек', 'очень', 'наш', 'работа', 'всё', 'должный']
Тема 7: ['год', 'это', 'весь', 'всё', 'свой', 'очень', 'человек', 'первый', 'ещё', 'говорить']
Тема 8: ['лотерея', 'год', 'государственный', 'рубль', 'срок', 'страна', 'весь', 'проведение', 'партия', 'число']
Тема 9: ['который', 'это', 'квартира', 'год', 'говорить', 'человек', 'весь', 'город', 'жильё', '

# SVD / LSA

In [ ]:
from sklearn.decomposition import TruncatedSVD

lsa_model = TruncatedSVD(n_components=15)
lsa_topic_matrix = lsa_model.fit_transform(test_vectorized)
lsa_components = lsa_model.components_
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lsa_components):
    print(f"Тема {topic_idx + 1}: ", end='')
    print(", ".join([feature_names[i] for i in topic.argsort()[-5:][::-1]]))

Тема 1: это, всё, свой, который, год
Тема 2: это, человек, который, россия, год
Тема 3: партия, год, новый, политический, организация
Тема 4: ребёнок, деньга, квартира, рязанов, семья
Тема 5: рязанов, квартира, договор, владимирович, людмила
Тема 6: партия, человек, рязанов, деньга, ребёнок
Тема 7: вера, русский, холодный, украина, говорить
Тема 8: животное, россия, страна, свой, президент
Тема 9: животное, вера, холодный, дикий, русский
Тема 10: это, украина, ребёнок, весь, палатка
Тема 11: деньга, год, человек, очень, украина
Тема 12: животное, это, подпись, казахстан, школа
Тема 13: подпись, киностудия, крым, фильм, всё
Тема 14: украина, свой, любовь, город, москва
Тема 15: подпись, кандидат, сбор, украина, встреча
